# Inference time measurement 

To be able to run this experimentation, [Grid2op](https://github.com/rte-france/Grid2Op) (1.7.1) and [LightSim2Grid](https://github.com/BDonnot/lightsim2grid) (0.7.0) are required. See the corresponding links for more information concerning their installation.

In [1]:
import pathlib
import numpy as np
from pprint import pprint
from matplotlib import pyplot as plt
from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from lips.utils import get_path

# TOC
- [Device Selection](#device)
- [Environment: `l2rpn_case14_sandbox`](#env_case14)
    - [Benchmark1](#benchmark1)
        - [Physics solver time](#14_solver_bench1)
    - [Benchmark2](#benchmark2)
        - [Physics solver time](#14_solver_bench2)
- [Environment: `l2rpn_neurips_2020_track1_small`](#env_nips)
    - [Benchmark1](#nips_benchmark1)
        - [Physics solver time](#nips_solver_bench1)
    - [Benchmark2](#nips_benchmark2)
        - [Physics solver time](#nips_solver_bench2)

## Select a device on which the model should be evaluated <a id="device"></a>
On the basis of GPU or CPU device, the results may be vary significantly. You can choose one of the following block in each kernel to using whether GPU or CPU device. 

### Using A GPU

In [2]:
import tensorflow as tf
device = "GPU"
gpus = tf.config.experimental.list_physical_devices(device)
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[3], device)
    except RuntimeError as e:
        # Visible devices must be set at program startup
        print(e)

### Using CPU

In [2]:
import os
device = "CPU"
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

### Show the list of used devices

In [ ]:
tf.config.experimental.get_visible_devices()

<h1 style="color:green">Environment : "l2rpn_case14_sandbox"</h1> <a id="env_case14"></a>

In [4]:
# indicate required paths
LIPS_PATH = pathlib.Path().resolve().parent # it is supposed that the notebook had run from getting_started folder
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_case14_sandbox"
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_case14_sandbox.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
TRAINED_MODEL_PATH = LIPS_PATH / "trained_models" / "powergrid"
EVALUATION_PATH = LIPS_PATH / "evaluation_results" / "PowerGrid"
LOG_PATH = LIPS_PATH / "lips_logs.log"

<h2 style="color:orange"> Benchmark1 </h2> <a id="benchmark1"></a>

In [5]:
benchmark1 = PowerGridBenchmark(benchmark_name="Benchmark1",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

### Physics solver time <a id="14_solver_bench1"></a>

Physics solver takes 0.02 miliseconds to solve a powerflow. However the physics solver time for the benchmark1 is computed as follows:
$$\underbrace{\#Samples}_\textrm{# situations} \times \underbrace{(3! + 1)}_\textrm{# topologies} \times \frac{\textrm{security analysis time}}{20}$$, where $20$ is number of the lines in this benchmark.

Do the Security Analysis. Source : https://github.com/BDonnot/lightsim2grid/blob/master/examples/security_analysis.py

In [6]:
import grid2op
from grid2op.Parameters import Parameters
from lightsim2grid import LightSimBackend, SecurityAnalysis

env_name = "l2rpn_case14_sandbox"

test = False
# Create the grid2op environment
param = Parameters()
param.NO_OVERFLOW_DISCONNECTION = True
env = grid2op.make(env_name,
                   backend=LightSimBackend(),
                   # ignore the protection, that are NOT simulated
                   # by the TimeSerie module !
                   param=param,
                   test=test)

# Run the environment on a scenario using the TimeSerie module
security_analysis = SecurityAnalysis(env)
security_analysis.add_all_n1_contingencies()
p_or, a_or, voltages = security_analysis.get_flows()
# the 3 lines above are the only lines you need to do to perform a security analysis !

computer = security_analysis.computer
print(f"For environment: {env_name} ({computer.nb_solved()} n-1 simulated)")
print(f"Total time spent in \"computer\" to solve everything: {1e3*computer.total_time():.1f}ms "
      f"({computer.nb_solved() / computer.total_time():.0f} pf / s), "
      f"{1000.*computer.total_time() / computer.nb_solved():.2f} ms / pf)")
print(f"\t - time to compute the coefficients to simulate line disconnection: {1e3*computer.preprocessing_time():.2f}ms")
print(f"\t - time to pre process Ybus: {1e3*computer.modif_Ybus_time():.2f}ms")
print(f"\t - time to perform powerflows: {1e3*computer.solver_time():.2f}ms "
      f"({computer.nb_solved() / computer.solver_time():.0f} pf / s, "
      f"{1000.*computer.solver_time() / computer.nb_solved():.2f} ms / pf)")
print(f"In addition, it took {1e3*computer.amps_computation_time():.2f} ms to retrieve the current "
      f"from the complex voltages (in total "
      f"{computer.nb_solved() / ( computer.total_time() + computer.amps_computation_time()):.1f} "
      "pf /s, "
      f"{1000.*( computer.total_time() + computer.amps_computation_time()) / computer.nb_solved():.2f} ms / pf)")

For environment: l2rpn_case14_sandbox (19 n-1 simulated)
Total time spent in "computer" to solve everything: 0.4ms (52145 pf / s), 0.02 ms / pf)
	 - time to compute the coefficients to simulate line disconnection: 0.01ms
	 - time to pre process Ybus: 0.03ms
	 - time to perform powerflows: 0.32ms (58712 pf / s, 0.02 ms / pf)
In addition, it took 0.01 ms to retrieve the current from the complex voltages (in total 50223.9 pf /s, 0.02 ms / pf)


In [7]:
nb_samples = int(1e6)
physics_solver_time = ((1e3*computer.total_time() / computer.nb_solved()) * nb_samples)/1e3
print(f"{physics_solver_time:.2f} s for {nb_samples} samples")

19.18 s for 1000000 samples


<h3 style="color:red"> Fully connected model </h3>

In [8]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

##### Load it from a saved model

In [9]:
LOAD_PATH = get_path(BASELINES_PATH, benchmark1)
tf_fc.restore(LOAD_PATH)

2022-06-08 13:44:17.566689: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


##### Augment the datasize to represent the realistic power network

In [10]:
def augment_data(benchmark, size=1e6):
    data_size = benchmark._test_dataset.size
    if size < data_size:
        raise ValueError("You cannot reduce the data size using this function")
    factor = int(size / data_size)
    
    for nm_, arr_ in benchmark._test_dataset.data.items():
        if nm_ == "line_status":
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=int)
        elif nm_ == "topo_vect":
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=bool)
        else:
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=np.float32)
        
    benchmark._test_dataset.size = int(size)
    return benchmark

In [11]:
benchmark1 = augment_data(benchmark1, size=nb_samples)

In [12]:
benchmark1._test_dataset.size

1000000

##### Evaluate it multiple times and return the inference times 

###### Run once

In [ ]:
tf_fc_metrics = benchmark1.evaluate_simulator(augmented_simulator=tf_fc,
                                              eval_batch_size=128,
                                              dataset="test",
                                              shuffle=False,
                                              save_path=None,
                                              save_predictions=False
                                             )

In [14]:
print(tf_fc_metrics["test"]["ML"]["TIME_INF"])
print(tf_fc_metrics["test"]["IndRed"]["TIME_INF"])

1.5277208059560508
17.477608849992976


###### Run multiple times

In [ ]:
NB_RUN = 5
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    tf_fc_metrics = benchmark1.evaluate_simulator(augmented_simulator=tf_fc,
                                                  eval_batch_size=128,
                                                  dataset="test",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )
    time_ml.append(tf_fc_metrics["test"]["ML"]["TIME_INF"])
    time_ind.append(tf_fc_metrics["test"]["IndRed"]["TIME_INF"])

The inference time should be reported per powerflow. So it should be devided by $10000$ which represents the size of the data in test data dataset. For the sake of comparison which physics solver, we would report it in miliseconds. So, it should be multiplied by $1000$. 

######  CPU

In [17]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / nb_samples) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / nb_samples) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.4f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.4f} ms/pf".format(device, inf_time_ML_per_pf))

Inference time from Industrial Readiness category and using CPU: 0.0176 ms/pf
Inference time from Machine Learning category and using CPU: 0.0015 ms/pf


In [18]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Inference time from Industrial Readiness category and using CPU: 17.61 s for 1000000 samples
Inference time from Machine Learning category and using CPU: 1.47 s for 1000000 samples


Computing the speed up 

In [19]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

Speed up factor from IndRed point of view using FullyConnected model and CPU is: 1.09
Speed up factor from ML point of view using FullyConnected model and CPU is: 13.06


###### GPU

In [14]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / 10000) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / 10000) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.2f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.2f} ms/pf".format(device, inf_time_ML_per_pf))

Inference time from Industrial Readiness category and using GPU: 1.42 ms/pf
Inference time from Machine Learning category and using GPU: 0.12 ms/pf


In [15]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Inference time from Industrial Readiness category and using GPU: 14.25 s for 1000000 samples
Inference time from Machine Learning category and using GPU: 1.25 s for 1000000 samples


Computing the speed up 

In [16]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

Speed up factor from IndRed point of view using FullyConnected model and GPU is: 1.25
Speed up factor from ML point of view using FullyConnected model and GPU is: 14.23


<h3 style="color:red"> LeapNet model </h3>

In [51]:
from lips.augmented_simulators.tensorflow_models import LeapNet
from lips.dataset.scaler import PowerGridScaler

leap_net = LeapNet(name="tf_leapnet",
                   bench_config_path=BENCH_CONFIG_PATH,
                   bench_config_name="Benchmark1",
                   sim_config_path=SIM_CONFIG_PATH / "tf_leapnet.ini",
                   sim_config_name="DEFAULT",
                   scaler=PowerGridScaler,
                   log_path=LOG_PATH)

Load the model for the convenience.

In [52]:
LOAD_PATH = get_path(BASELINES_PATH, benchmark1)
leap_net.restore(path=LOAD_PATH)

##### Evaluate it multiple times

In [ ]:
NB_RUN = 10
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    leapnet_metrics = benchmark1.evaluate_simulator(augmented_simulator=leap_net,
                                                    eval_batch_size=128,
                                                    dataset="test",
                                                    shuffle=False,
                                                    save_path=None,
                                                    save_predictions=False
                                                 )
    time_ml.append(leapnet_metrics["test"]["ML"]["TIME_INF"])
    time_ind.append(leapnet_metrics["test"]["IndRed"]["TIME_INF"])

######  CPU

In [ ]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / 10000) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / 10000) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.2f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.2f} ms/pf".format(device, inf_time_ML_per_pf))

In [ ]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Computing the speed up 

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

###### GPU

In [ ]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / 10000) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / 10000) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.2f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.2f} ms/pf".format(device, inf_time_ML_per_pf))

In [ ]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Computing the speed up 

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

<h2 style="color:orange"> Benchmark2</h2> <a id="benchmark2"></a>

In [ ]:
benchmark2 = PowerGridBenchmark(benchmark_name="Benchmark2",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

### Physics solver time <a id="14_solver_bench2"></a>

Apart of first benchmark (security analysis) where the inverse of `Y_bus` matrix could be computed once and the observatiosn could be generated in parallel, the other two benchmarks require to compute this matrix inversion for each flow computation. Hence, to be able to compute the physics solver time, we can use the following times:

$$ \textrm{_timer_preproc} + \textrm{_timer_solver} + \textrm{_timer_postproc}$$

The details explanations for each of these steps are provided [here](https://docs.google.com/document/d/1MWj1NUj4hbPuMMzu0ty9Md_qk9mnLz88chEOi3TwJJU/edit#).

In the next cell we compute the flow over 1000 observations, and finally we report the time required for one powerflow to be comptued.

In [ ]:
NB_RUN = 10
benchmark2_tmp = PowerGridBenchmark(benchmark_name="Benchmark2",
                                    benchmark_path=None,
                                    load_data_set=False,
                                    log_path=LOG_PATH,
                                    config_path=BENCH_CONFIG_PATH
                                   )
time_physics_solver = list()
for i in range(NB_RUN):
    benchmark2_tmp.generate(nb_sample_train=int(1),
                            nb_sample_val=int(1),
                            nb_sample_test=int(10000),
                            nb_sample_test_ood_topo=int(1),
                           )
    simulator = benchmark2_tmp.test_simulator
    total_time = simulator._timer_preproc + simulator._timer_solver + simulator._timer_postproc
    #total_time = benchmark2_tmp.test_simulator._simulator.backend._timer_postproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_preproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_solver
    time_physics_solver.append(total_time)

In [ ]:
physics_solver_time = np.mean(time_physics_solver)
print("Physics solver computation time for benchmark 2 over {} runs is: {} s".format(NB_RUN, physics_solver_time))
print("Physics solver computation time for benchmark 2 over {} runs and for one powerflow is: {} ms".format(NB_RUN, (physics_solver_time/10000)*1000))

In [ ]:
physics_solver_time = 1.9429454155266286

<h3 style="color:red"> Fully Connected model</h3>

In [ ]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

In [ ]:
LOAD_PATH = get_path(BASELINES_PATH, benchmark2)
tf_fc.restore(LOAD_PATH)

##### Evaluate it multiple times and return the inference times 

In [ ]:
NB_RUN = 10
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    tf_fc_metrics = benchmark2.evaluate_simulator(augmented_simulator=tf_fc,
                                                  eval_batch_size=128,
                                                  dataset="test",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )
    time_ml.append(tf_fc_metrics["test"]["ML"]["TIME_INF"])
    time_ind.append(tf_fc_metrics["test"]["IndRed"]["TIME_INF"])

######  CPU

In [ ]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / nb_samples) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / nb_samples) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.4f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.4f} ms/pf".format(device, inf_time_ML_per_pf))

In [ ]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Computing the speed up 

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

###### GPU

In [ ]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / nb_samples) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / nb_samples) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.4f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.4f} ms/pf".format(device, inf_time_ML_per_pf))

In [ ]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Computing the speed up 

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

<h3 style="color:red"> LeapNet model </h3>

In [ ]:
from lips.augmented_simulators.tensorflow_models import LeapNet
from lips.dataset.scaler import PowerGridScaler

leap_net = LeapNet(name="tf_leapnet",
                   bench_config_path=BENCH_CONFIG_PATH,
                   bench_config_name="Benchmark1",
                   sim_config_path=SIM_CONFIG_PATH / "tf_leapnet.ini",
                   sim_config_name="DEFAULT",
                   scaler=PowerGridScaler,
                   log_path=LOG_PATH)

Load the model for the convenience.

In [ ]:
LOAD_PATH = get_path(BASELINES_PATH, benchmark2)
leap_net.restore(path=LOAD_PATH)

##### Evaluate it multiple times

In [ ]:
NB_RUN = 10
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    leapnet_metrics = benchmark2.evaluate_simulator(augmented_simulator=leap_net,
                                                    eval_batch_size=128,
                                                    dataset="all",
                                                    shuffle=False,
                                                    save_path=None,
                                                    save_predictions=False
                                                 )
    time_ml.append(leapnet_metrics["test"]["ML"]["TIME_INF"])
    time_ind.append(leapnet_metrics["test"]["IndRed"]["TIME_INF"])

######  CPU

In [ ]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / nb_samples) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / nb_samples) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.4f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.4f} ms/pf".format(device, inf_time_ML_per_pf))

In [ ]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Computing the speed up 

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

###### GPU

In [ ]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / nb_samples) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / nb_samples) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.4f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.4f} ms/pf".format(device, inf_time_ML_per_pf))

In [ ]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Computing the speed up 

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

<h1 style="color:green">Environment : "l2rpn_neurips_2020_track1_small"</h1> <a id="env_nips"></a>

In [20]:
# indicate required paths
LIPS_PATH = pathlib.Path().resolve().parent # it is supposed that the notebook had run from getting_started folder
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_neurips_2020_track1_small"
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_neurips_2020_track1_small.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
TRAINED_MODEL_PATH = LIPS_PATH / "trained_models" / "powergrid"
EVALUATION_PATH = LIPS_PATH / "evaluation_results" / "PowerGrid"
LOG_PATH = LIPS_PATH / "lips_logs.log"

<h2 style="color:orange"> Benchmark1 </h2> <a id="nips_benchmark1"></a>

In [21]:
benchmark1 = PowerGridBenchmark(benchmark_name="Benchmark1",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

### Physics solver time <a id="nips_solver_bench1"></a>

Physics solver takes 0.02 miliseconds to solve a powerflow. However the physics solver time for the benchmark1 is computed as follows:
$$\underbrace{\# samples}_\textrm{# situations} \times \underbrace{(4! + 1)}_\textrm{# topologies} \times \frac{\textrm{security analysis time}}{58}$$, where $58$ is number of the lines for this environment.

In [22]:
import grid2op
from grid2op.Parameters import Parameters
from lightsim2grid import LightSimBackend, SecurityAnalysis

env_name = "l2rpn_neurips_2020_track1_small"

test = False
# Create the grid2op environment
param = Parameters()
param.NO_OVERFLOW_DISCONNECTION = True
env = grid2op.make(env_name,
                   backend=LightSimBackend(),
                   # ignore the protection, that are NOT simulated
                   # by the TimeSerie module !
                   param=param,
                   test=test)

# Run the environment on a scenario using the TimeSerie module
security_analysis = SecurityAnalysis(env)
security_analysis.add_all_n1_contingencies()
p_or, a_or, voltages = security_analysis.get_flows()
# the 3 lines above are the only lines you need to do to perform a security analysis !

computer = security_analysis.computer
print(f"For environment: {env_name} ({computer.nb_solved()} n-1 simulated)")
print(f"Total time spent in \"computer\" to solve everything: {1e3*computer.total_time():.1f}ms "
      f"({computer.nb_solved() / computer.total_time():.0f} pf / s), "
      f"{1000.*computer.total_time() / computer.nb_solved():.2f} ms / pf)")
print(f"\t - time to compute the coefficients to simulate line disconnection: {1e3*computer.preprocessing_time():.2f}ms")
print(f"\t - time to pre process Ybus: {1e3*computer.modif_Ybus_time():.2f}ms")
print(f"\t - time to perform powerflows: {1e3*computer.solver_time():.2f}ms "
      f"({computer.nb_solved() / computer.solver_time():.0f} pf / s, "
      f"{1000.*computer.solver_time() / computer.nb_solved():.2f} ms / pf)")
print(f"In addition, it took {1e3*computer.amps_computation_time():.2f} ms to retrieve the current "
      f"from the complex voltages (in total "
      f"{computer.nb_solved() / ( computer.total_time() + computer.amps_computation_time()):.1f} "
      "pf /s, "
      f"{1000.*( computer.total_time() + computer.amps_computation_time()) / computer.nb_solved():.2f} ms / pf)")

/home/ubuntu/SYSTEMX/milad/venv/lips_milad/lib/python3.8/site-packages/lightsim2grid/gridmodel/_aux_add_trafo.py:65: UserWarning: There were some Nan in the pp_net.trafo["tap_step_degree"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_step_degree\"], they have been replaced by 0")
/home/ubuntu/SYSTEMX/milad/venv/lips_milad/lib/python3.8/site-packages/lightsim2grid/gridmodel/_aux_add_slack.py:113: UserWarning: We found either some slack coefficient to be < 0. or they were all 0.We set them all to 1.0 to avoid such issues
  warnings.warn("We found either some slack coefficient to be < 0. or they were all 0."


For environment: l2rpn_neurips_2020_track1_small (58 n-1 simulated)
Total time spent in "computer" to solve everything: 1.9ms (30576 pf / s), 0.03 ms / pf)
	 - time to compute the coefficients to simulate line disconnection: 0.02ms
	 - time to pre process Ybus: 0.15ms
	 - time to perform powerflows: 1.70ms (34105 pf / s, 0.03 ms / pf)
In addition, it took 0.08 ms to retrieve the current from the complex voltages (in total 29275.9 pf /s, 0.03 ms / pf)


In [23]:
nb_samples = int(1e6)
physics_solver_time = ((1e3*computer.total_time() / computer.nb_solved()) * nb_samples)/1e3
print(f"{physics_solver_time:.2f} s for {nb_samples} samples")

32.71 s for 1000000 samples


<h3 style="color:red"> Fully connected model </h3>

In [24]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

##### Load it from a saved model

In [25]:
LOAD_PATH = get_path(TRAINED_MODEL_PATH, benchmark1)
tf_fc.restore(LOAD_PATH)

##### Augment the datasize to represent the realistic power network

In [26]:
def augment_data(benchmark, size=1e6):
    data_size = benchmark._test_dataset.size
    if size < data_size:
        raise ValueError("You cannot reduce the data size using this function")
    factor = int(size / data_size)
    
    for nm_, arr_ in benchmark._test_dataset.data.items():
        if nm_ == "line_status":
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=int)
        elif nm_ == "topo_vect":
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=bool)
        else:
            benchmark._test_dataset.data[nm_] = np.asarray(np.tile(arr_, (factor, 1)), dtype=np.float32)
        
    benchmark._test_dataset.size = int(size)
    return benchmark

In [27]:
benchmark1 = augment_data(benchmark1, size=nb_samples)

In [28]:
benchmark1._test_dataset.size

1000000

##### Evaluate it multiple times and return the inference times 

In [ ]:
NB_RUN = 5
time_ml = list()
time_ind = list()
for i in range(NB_RUN):
    tf_fc_metrics = benchmark1.evaluate_simulator(augmented_simulator=tf_fc,
                                                  eval_batch_size=128,
                                                  dataset="test",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )
    time_ml.append(tf_fc_metrics["test"]["ML"]["TIME_INF"])
    time_ind.append(tf_fc_metrics["test"]["IndRed"]["TIME_INF"])

The inference time should be reported per powerflow. So it should be devided by $10000$ which represents the size of the data in test data dataset. For the sake of comparison which physics solver, we would report it in miliseconds. So, it should be multiplied by $1000$. 

######  CPU

In [31]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / nb_samples) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / nb_samples) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.4f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.4f} ms/pf".format(device, inf_time_ML_per_pf))

Inference time from Industrial Readiness category and using CPU: 0.0094 ms/pf
Inference time from Machine Learning category and using CPU: 0.0030 ms/pf


In [32]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Inference time from Industrial Readiness category and using CPU: 9.40 s for 1000000 samples
Inference time from Machine Learning category and using CPU: 3.03 s for 1000000 samples


Computing the speed up 

In [33]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

Speed up factor from IndRed point of view using FullyConnected model and CPU is: 3.48
Speed up factor from ML point of view using FullyConnected model and CPU is: 10.79


###### GPU

In [34]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / nb_samples) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / nb_samples) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.4f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.4f} ms/pf".format(device, inf_time_ML_per_pf))

Inference time from Industrial Readiness category and using GPU: 0.0075 ms/pf
Inference time from Machine Learning category and using GPU: 0.0034 ms/pf


In [28]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Inference time from Industrial Readiness category and using GPU: 7.52 s for 1000000 samples
Inference time from Machine Learning category and using GPU: 3.39 s for 1000000 samples


Computing the speed up 

In [29]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

Speed up factor from IndRed point of view using FullyConnected model and GPU is: 4.36
Speed up factor from ML point of view using FullyConnected model and GPU is: 9.65


<h2 style="color:orange"> Benchmark2 </h2> <a id="nips_benchmark2"></a>

In [ ]:
benchmark2 = PowerGridBenchmark(benchmark_name="Benchmark2",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

### Physics solver time <a id="nips_solver_bench2"></a>

Apart of first benchmark (security analysis) where the inverse of `Y_bus` matrix could be computed once and the observatiosn could be generated in parallel, the other two benchmarks require to compute this matrix inversion for each flow computation. Hence, to be able to compute the physics solver time, we can use the following times:

$$ \textrm{_timer_preproc} + \textrm{_timer_solver} + \textrm{_timer_postproc}$$

The details explanations for each of these steps are provided [here](https://docs.google.com/document/d/1MWj1NUj4hbPuMMzu0ty9Md_qk9mnLz88chEOi3TwJJU/edit#).

In the next cell we compute the flow over 1000 observations, and finally we report the time required for one powerflow to be comptued.

In [ ]:
NB_RUN = 10
benchmark2_tmp = PowerGridBenchmark(benchmark_name="Benchmark2",
                                    benchmark_path=None,
                                    load_data_set=False,
                                    log_path=LOG_PATH,
                                    config_path=BENCH_CONFIG_PATH
                                   )
time_physics_solver = list()
for i in range(NB_RUN):
    benchmark2_tmp.generate(nb_sample_train=int(1),
                            nb_sample_val=int(1),
                            nb_sample_test=int(10000),
                            nb_sample_test_ood_topo=int(1),
                           )
    simulator = benchmark2_tmp.test_simulator
    total_time = simulator._timer_preproc + simulator._timer_solver + simulator._timer_postproc
    #total_time = benchmark2_tmp.test_simulator._simulator.backend._timer_postproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_preproc + \
    #             benchmark2_tmp.test_simulator._simulator.backend._timer_solver
    time_physics_solver.append(total_time)

In [ ]:
physics_solver_time = np.mean(time_physics_solver)
print("Physics solver computation time for benchmark 2 over {} runs is: {} s".format(NB_RUN, physics_solver_time))
print("Physics solver computation time for benchmark 2 over {} runs and for one powerflow is: {} ms".format(NB_RUN, (physics_solver_time/10000)*1000))

In [ ]:
physics_solver_time = 2.8298422046704217

<h3 style="color:red"> Fully Connected model </h3>

In [ ]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark1",
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

In [ ]:
LOAD_PATH = get_path(TRAINED_MODEL_PATH, benchmark2)
tf_fc.restore(LOAD_PATH)

##### Evaluate it multiple times and return the inference times 

In [ ]:
NB_RUN = 10
time_ml = list()
time_ind = list()
for i in range(10):
    tf_fc_metrics = benchmark2.evaluate_simulator(augmented_simulator=tf_fc,
                                                  eval_batch_size=128,
                                                  dataset="all",
                                                  shuffle=False,
                                                  save_path=None,
                                                  save_predictions=False
                                                 )
    time_ml.append(tf_fc_metrics["test"]["ML"]["TIME_INF"])
    time_ind.append(tf_fc_metrics["test"]["IndRed"]["TIME_INF"])

######  CPU

In [ ]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / nb_samples) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / nb_samples) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.4f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.4f} ms/pf".format(device, inf_time_ML_per_pf))

In [ ]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Computing the speed up 

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))

###### GPU

In [ ]:
inf_time_IndRed_per_pf = (np.mean(time_ind) / nb_samples) * 1000
inf_time_ML_per_pf = (np.mean(time_ml) / nb_samples) * 1000
print("Inference time from Industrial Readiness category and using {}: {:.4f} ms/pf".format(device, inf_time_IndRed_per_pf))
print("Inference time from Machine Learning category and using {}: {:.4f} ms/pf".format(device, inf_time_ML_per_pf))

In [ ]:
inf_time_IndRed_total = np.mean(time_ind)
inf_time_ML_per_total = np.mean(time_ml)
print("Inference time from Industrial Readiness category and using {}: {:.2f} s for {} samples".format(device, inf_time_IndRed_total, benchmark1._test_dataset.size))
print("Inference time from Machine Learning category and using {}: {:.2f} s for {} samples".format(device, inf_time_ML_per_total, benchmark1._test_dataset.size))

Computing the speed up 

In [ ]:
print("Speed up factor from IndRed point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_IndRed_total)))
print("Speed up factor from ML point of view using FullyConnected model and {} is: {:.2f}".format(device, (physics_solver_time / inf_time_ML_per_total)))